In [84]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [85]:
%matplotlib inline
plt.rcParams['figure.figsize'] = (10,6)

In [86]:
#df = pd.read_excel('./data_cleaned_copy.xlsx', sheet_name='data_cleaned', na_values=['#NAME?'], engine='openpyxl' )
df = pd.read_csv('data_cleaned.csv')

In [87]:
df.head()

,Job_position,Company,Location,Salary,requirements,rating,experience,posting_frequency
0,Junior Software Developer,Gather Network,"Urban Estate Gurgaon, Haryana","20,000 a month",We are actively looking for a few freshers who...,-99.0,na,1.0
1,Big Data Developer,Agile Placement,"Pune, Maharashtra","5,00,000 - 14,00,000 a year",_Strong knowledge of programming and scripting...,-99.0,Total work: 6 years,1.0
2,Data Scientist,Client of PlaceElements HR,"Thiruvananthapuram, Kerala","5,00,000 - 10,00,000 a year",Experience in working closely with data analys...,-99.0,Python: Algorithms: Statistics: 1 yearData Sci...,1.0
3,Software Engineer - Machine Learning Engineer,Siemens Technology and Services Private Limited,"Pune, Maharashtra",na,Experience with machine learning architectures...,4.0,na,1.0
4,Machine Learning (Fullstack) at Sadhashiva Nag...,Teqlinx Software Solutions LLC,"Bengaluru, Karnataka","30,00,000 a year",• Extremely strong programming background – da...,-99.0,na,1.0


In [88]:
# to calculate max and min Salary per annum

yearly_min = {}
yearly_max = {}

def Salary(df):
    
    for i in range(0, len(df)):
        
        if df['Salary'][i] == '-999':
            yearly_min[i] = 0
            yearly_max[i] = 0
            
        if 'a year' in df['Salary'][i]:
            sal_min = df['Salary'][i].split('-')[0].replace('a year','').replace(',','')
            yearly_min[i] = int(sal_min)
            
            try:
                sal_max = df['Salary'][i].split('-')[1].replace('a year','').replace(',','')
                yearly_max[i] = int(sal_max)
                
            # if only single value present will be stored in both max and min, so the average comes accuate
            except:
                sal_max = df['Salary'][i].split('-')[0].replace('a year','').replace(',','')
                yearly_max[i] = int(sal_max)
            
        if 'a month' in df['Salary'][i]:
            sal_min = df['Salary'][i].split('-')[0].replace('a month','').replace(',','')
            yearly_min[i] = int(sal_min) * 12
            
            try:
                sal_max = df['Salary'][i].split('-')[1].replace('a month','').replace(',','')
                yearly_max[i] = int(sal_max) * 12    
                
            # if only single value present will be stored in both max and min, so the average comes accuate
            except:
                sal_max = df['Salary'][i].split('-')[0].replace('a month','').replace(',','')
                yearly_max[i] = int(sal_max) * 12
                
Salary(df)

In [89]:
# min, max and avg Salary columns
df['min_Salary'] = pd.DataFrame(yearly_min.values(), index= yearly_min.keys())
df['max_Salary'] = pd.DataFrame(yearly_max.values(), index= yearly_max.keys())
df['avg_yearly_sal'] = ( df['min_Salary'] + df['max_Salary'] )/2
df['monthly_Salary'] = df['avg_yearly_sal']/12.

In [90]:
df['avg_yearly_sal'].fillna(0, inplace=True)
df['min_Salary'].fillna(0, inplace=True)
df['max_Salary'].fillna(0, inplace=True)
df['monthly_Salary'].fillna(0, inplace=True)

Lets just drop these as we got our target column

In [91]:
df.drop('max_Salary', axis=1, inplace=True)
df.drop('min_Salary', axis=1, inplace=True)
df.drop('monthly_Salary', axis=1, inplace=True)

we can divide the annual Salary into 6 differrent categories

In [92]:
df['income_cat'] = pd.cut(df['avg_yearly_sal'], bins=[-999, 0, 50000, 100000, 500000, 1000000, 2500000, np.inf], labels=[-1, 1, 2, 3, 4, 5, 6])

In [93]:
df.drop('Salary', axis=1, inplace=True)

In [167]:
freq = df.groupby(['income_cat', 'rating'])['avg_yearly_sal'].mean()

In [165]:
x = df[(df['rating'] >0) & (df['avg_yearly_sal'] == 0)]['income_cat'].map(freq)

c:\python\python38\lib\site-packages\pandas\io\formats\format.py:1405: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
  for val, m in zip(values.ravel(), mask.ravel())


15      77271.49472
69      77271.49472
142     77271.49472
170     77271.49472
203     77271.49472
250     77271.49472
259     77271.49472
560     77271.49472
727     77271.49472
958     77271.49472
1010    77271.49472
Name: income_cat, dtype: category
Categories (7, float64): [7.727149e+04 < 2.922222e+04 < 8.500000e+04 < 2.939141e+05 < 7.242970e+05 < 1.422497e+06 < 4.725950e+06]

In [99]:
df.loc[x.index,'avg_yearly_sal'] = x.values

In [170]:
df[(df['rating']<0) & (df['avg_yearly_sal']>0)]

,Job_position,Company,Location,requirements,rating,experience,posting_frequency,avg_yearly_sal,income_cat,net_experience,...,api,js,cleaning,algorithms,architecture,information,jquery,bootstrap,react,apis
0,na,Gather Network,"Urban Estate Gurgaon, Haryana",blockchain,-99.0,na,1.0,240000.0,3,-99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,na,Agile Placement,"Pune, Maharashtra",java spark python r,-99.0,Total work: 6 years,1.0,950000.0,4,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,na,Client of PlaceElements HR,"Thiruvananthapuram, Kerala",,-99.0,Python: Algorithms: Statistics: 1 yearData Sci...,1.0,750000.0,4,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,na,Teqlinx Software Solutions LLC,"Bengaluru, Karnataka",structures,-99.0,na,1.0,3000000.0,6,-99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,java,www.yansisofsol.com,"Hyderabad, Telangana",cleaning,-99.0,na,1.0,342000.0,3,-99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,php,Multiline Technolabs,"Ahmedabad, Gujarat",jquery,-99.0,na,15.0,240000.0,3,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1119,java,megna super It solutions private limited,"Hyderabad, Telangana",java,-99.0,na,1.0,540000.0,4,-99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1120,na,Welkin Smartcard Business pvt. Ltd.,"Bhopal, Madhya Pradesh",mvc,-99.0,na,1.0,270000.0,3,-99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1121,stack,Perfect Plan B Private Limited - Edtech Indust...,"Faridabad, Haryana",stack,-99.0,na,1.0,360000.0,3,-99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [185]:
freq = df[(df['rating']>0) & (df['avg_yearly_sal']>0)].groupby('rating')['income_cat'].median()

DataError: No numeric types to aggregate

In [184]:
freq

income_cat
-1    3.90
1      NaN
2     3.70
3     3.85
4     3.30
5     3.90
6     4.30
Name: rating, dtype: float64

In [183]:
df[(df['rating']<0) & (df['avg_yearly_sal']>0)]['rating'].map(freq).unique()

array([nan])

In [182]:
df[(df['rating']<0) & (df['avg_yearly_sal']>0)]

,Job_position,Company,Location,requirements,rating,experience,posting_frequency,avg_yearly_sal,income_cat,net_experience,...,api,js,cleaning,algorithms,architecture,information,jquery,bootstrap,react,apis
0,na,Gather Network,"Urban Estate Gurgaon, Haryana",blockchain,-99.0,na,1.0,240000.0,3,-99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,na,Agile Placement,"Pune, Maharashtra",java spark python r,-99.0,Total work: 6 years,1.0,950000.0,4,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,na,Client of PlaceElements HR,"Thiruvananthapuram, Kerala",,-99.0,Python: Algorithms: Statistics: 1 yearData Sci...,1.0,750000.0,4,2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,na,Teqlinx Software Solutions LLC,"Bengaluru, Karnataka",structures,-99.0,na,1.0,3000000.0,6,-99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,java,www.yansisofsol.com,"Hyderabad, Telangana",cleaning,-99.0,na,1.0,342000.0,3,-99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1117,php,Multiline Technolabs,"Ahmedabad, Gujarat",jquery,-99.0,na,15.0,240000.0,3,6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1119,java,megna super It solutions private limited,"Hyderabad, Telangana",java,-99.0,na,1.0,540000.0,4,-99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1120,na,Welkin Smartcard Business pvt. Ltd.,"Bhopal, Madhya Pradesh",mvc,-99.0,na,1.0,270000.0,3,-99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1121,stack,Perfect Plan B Private Limited - Edtech Indust...,"Faridabad, Haryana",stack,-99.0,na,1.0,360000.0,3,-99,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [100]:
#df[df['rating'] > 0].groupby('income_cat')['rating'].mean()

In [101]:
#freq = df[df['rating'] > 0].groupby('income_cat')['rating'].mean()

In [102]:
#freq

Experience is mentioned in both requirements and experience so we will collect them all and save it in a column of experience 

Some of these requirements mention experienced  

In [103]:
net_experience = []
for i in df.experience:
    temp=[]
    for word in i.split():
        if word.isdigit():
            temp.append(word)
    if temp:
        temp.sort(reverse=True)
        net_experience.append(temp[0])
    else:
        net_experience.append(-99)
df['net_experience'] = net_experience

In [104]:
df['net_experience'] = df['net_experience'].astype('int32')

In [105]:
net_experience = []
for i in df.requirements:
    temp=[]
    for word in i.split():
        if word.isdigit():
            temp.append(word)
    if temp:
        temp.sort(reverse=True)
        net_experience.append(temp[0])
    else:
        net_experience.append(-99)
df['exp2'] = net_experience

Removing unwanted values from experience column

In [106]:
for p in ['²', '0080091', '2020', '2024', '2019', '90', '88', '32', '48', '40', '50', '24']:
    df['exp2'] = df['exp2'].apply(lambda x: str(x).replace(p,'-99'))

In [107]:
df['exp2'] = df['exp2'].astype('int32')

where experience required is mentioned in <b>requirements</b> column but missing in <b>experience</b> column 

In [108]:
df['net_experience'] = df['net_experience'].where((df['net_experience']>0), df['exp2'])
df.drop('exp2', axis=1, inplace=True)

In [109]:
df.iloc[[188, 294, 390, 723]]['experience']

188    Higher Secondary(12th Pass)
294    Higher Secondary(12th Pass)
390    Higher Secondary(12th Pass)
723    Higher Secondary(12th Pass)
Name: experience, dtype: object

In [110]:
df.iloc[[14, 111, 122, 362, 749]]['requirements']

14     About the Internship: Selected intern's day-to...
111    Full stack developer intern who could work on ...
122    We are Hiring for Junior Software Developer In...
362    AGNIK is hiring a Data Science Intern with som...
749    We are seeking a Web developer Intern responsi...
Name: requirements, dtype: object

Looks like Some openings require no experience

In [111]:
df['net_experience'][[188, 294, 390, 723, 14, 111, 122, 362, 749]] = 0

<ipython-input-111-2cd56dcae9cc>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['net_experience'][[188, 294, 390, 723, 14, 111, 122, 362, 749]] = 0


Educational criteria mentioned by these companies can also be useful

In [112]:
def education_level(data):
    if 'bachelor' in data.replace('year',' ').replace("'",' ').lower().split():
        return 'bachelor'
    if 'secondary' in data.replace('year',' ').replace('(',' ').replace("'",' ').lower().split():
        return 'secondary'
    if 'master' in data.replace('year',' ').replace("'",' ').lower().split():
        return 'masters'

In [113]:
df['education_level'] = df['experience'].map(education_level)
df['education_level'].fillna('na',inplace=True)

As the categories of seniority is only jr, senior or na, we can one hot encode them

In [114]:
df = pd.concat([df, pd.get_dummies(df['education_level'])], axis=1)

Seniority of these job positions cal also be useful

In [115]:
def seniority(title):
    title = str(title) 
    if 'ii' in title.lower().split() or 'director' in title.lower().split() or 'specialist' in title.lower().split() or 'professional' in title.lower().split() or 'sr.' in title.lower().split() or 'senior' in title.lower().split():
        return 'senior'
    elif 'i' in title.lower().split() or 'associate' in title.lower().split() or 'junior' in title.lower().split() or 'jr' in title.lower().split()  or 'jr.' in title.lower().split() or 'trainee' in title.lower().split() or 'intern' in title.lower().split() or 'jr.' in title.lower().split():
        return 'jr'
    else:
        return 'na'

In [116]:
# to calculate the seniority of the position applying for
df['job_title'] = df['Job_position'].apply(seniority)

For encoding rank transforamtion, label encoding, frequency encoding were applied but they had very weak correlation with avg_year_Salary <br>
as the categories of seniority is only jr, senior or na, we can one hot encode them

In [117]:
df = pd.concat([df, pd.get_dummies(df['job_title'])], axis=1)

Upon analyzing the requirements column following are the most popular professions <br>
lets store their frequencies

In [118]:
def calc_jobs(data):
    data = data.lower().replace(' ', '')
    if 'machinelearning' in data:
        return 'machine learning'
    
    if 'datascientist' in data:
        return 'data scientist'
    
    if 'softwaredeveloper' in data:
        return 'software developer'
    
    if 'softwareengineer' in data:
        return 'software engineer'
    
    if 'deeplearning' in data:
        return 'deep learning'

In [119]:
df['popular_profession'] = df['requirements'].apply(calc_jobs)


In [120]:
df['popular_profession'] = df['popular_profession'].fillna('na')

In [121]:
df = pd.concat([df, pd.get_dummies(df['popular_profession'])], axis=1)

In [122]:
# we can split the location column and get the state 
df['State'] = df['Location'].apply(lambda x: x.split(', ')[1] if len(x.split()) > 1 else x)

In [123]:
len(df['State'].unique())

29

We can one hot encode these States values

In [124]:
df = pd.concat([df, pd.get_dummies(df['State'])], axis=1)

In [125]:
# Some companies have multiple job openings this could be useful
df['Company'].value_counts().head()

Larsen & Toubro Infotech Limited    9
Accenture                           8
Shaw Academy                        7
ANI Calls India Private Limited     6
JPMorgan Chase Bank, N.A.           5
Name: Company, dtype: int64

In [126]:
job_openings = df['Company'].value_counts()

In [127]:
df['job_openings'] = df['Company'].map(job_openings)

In [128]:
df['requirements'].fillna('', inplace = True)

c:\python\python38\lib\site-packages\pandas\core\series.py:4433: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [129]:
df['job_descr_len'] = df['requirements'].apply(lambda x: 0 if not x else len(x))

In [130]:
df.head()

,Job_position,Company,Location,requirements,rating,experience,posting_frequency,avg_yearly_sal,income_cat,net_experience,...,Punjab,Rajasthan,Remote,Tamil Nadu,Telangana,Uttar Pradesh,Uttarakhand,West Bengal,job_openings,job_descr_len
0,Junior Software Developer,Gather Network,"Urban Estate Gurgaon, Haryana",We are actively looking for a few freshers who...,-99.0,na,1.0,2.400000e+05,3,-99,...,0,0,0,0,0,0,0,0,1,156
1,Big Data Developer,Agile Placement,"Pune, Maharashtra",_Strong knowledge of programming and scripting...,-99.0,Total work: 6 years,1.0,9.500000e+05,4,6,...,0,0,0,0,0,0,0,0,1,158
2,Data Scientist,Client of PlaceElements HR,"Thiruvananthapuram, Kerala",Experience in working closely with data analys...,-99.0,Python: Algorithms: Statistics: 1 yearData Sci...,1.0,7.500000e+05,4,2,...,0,0,0,0,0,0,0,0,1,156
3,Software Engineer - Machine Learning Engineer,Siemens Technology and Services Private Limited,"Pune, Maharashtra",Experience with machine learning architectures...,4.0,na,1.0,8.988636e+04,-1,-99,...,0,0,0,0,0,0,0,0,4,149
4,Machine Learning (Fullstack) at Sadhashiva Nag...,Teqlinx Software Solutions LLC,"Bengaluru, Karnataka",• Extremely strong programming background – da...,-99.0,na,1.0,3.000000e+06,6,-99,...,0,0,0,0,0,0,0,0,1,160


### Analyzing Job skills

As due to covid-19 many people working in the industry have lost their jobs, and according to news articles the skill demand for job industry 
is also changing, lets take a look at the skills, in demand in the job industry 

In [131]:
from nltk.tokenize import word_tokenize

In [132]:
requirements = df['requirements']

In [133]:
requirements = list(filter(None, requirements))

In [134]:
# split punctuation 
for p in ['-','(',')','.','/']:
    job_descr = []
    for i in range(0, len(requirements)):
        c = requirements[i].split(p)
        for x in c:
            x.replace('.',' ')
            job_descr.append(x)

In [135]:
# Remove punctuation and convert to lower case
for x in range(0,len(job_descr)):
    for p in ['.', '-', ')', '(', '…', ',', ':', "'"]:
        job_descr[x] = job_descr[x].replace(p,' ')
    job_descr[x] = job_descr[x].lower()       
    

In [136]:
# analyzing keywords from custom keyword list
f = open("skills.txt","r",) 
skills=[]
for x in f:
    skills.append(x)
f.close()

In [137]:
for i in skills:
    skills = i.split(',')

In [138]:
for i in range(0, len(skills)):
    skills[i] = skills[i].replace(' ','')
    skills[i] = skills[i].lower()

In [139]:
print(skills)

['html', 'css', 'c', 'debugging', 'git', 'graphic', 'database', 'java', 'javascript', 'nodejs', 'apis', 'r', 'ruby', 'php', 'net', 'c#', 'jquery', 'python', 'perl', 'react', 'reactjs', 'ux', 'ui', 'testing', 'analyst', 'springboot', 'jpa', 'microservice', 'postman', 'rest', 'api', 'angular', 'azure', 'aws', 'cloud', 'debugging', 'btech', 'cs', 'idbc', 'porting', 'porting', 'vive', 'go', 'playstation', 'medicine', 'food', 'chromium', 'nodejs', 'js', 'net', 'server', 'architecture', 'mobile', 'b2b', 'healthcare', 'security', 'sql', 'mvc', 'asp', 'version', 'typescript', 'sass', 'pwas', 'ios', 'androis', 'html', 'python.node', 'rdbms', 'mysql', 'script', 'query', 'mongo', 'oops', 'os', 'restful', 'app', 'xamarin', 'financial', 'shell', 'unix', 'script', 'powershell', 'linux', 'game', 'gui', 'unity', 'ai', 'jquery', 'iot', 'freelance', 'bootstrap', 'word', 'excel', 'swift', 'salesforce', 'graphic', 'github', 'flutter', 'c++', 'c#', 'docker', 'stack', 'bug', 'lravel', 'flux', 'redux', 'nlp'

Processing requirements on the basis whether a skill is present in that row

In [140]:
df['requirements'] = df['requirements'].apply(lambda x: ' '.join([word for word in x.lower().split() if word in (skills)]))
df['requirements'].replace(to_replace='', value='na', inplace=True)

c:\python\python38\lib\site-packages\pandas\core\series.py:4479: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [141]:
processed_text = word_tokenize(str(job_descr))

In [142]:
# to calculate the frequency of a particular skill mentioned in job description
def calc_skill_freq(data):
    skill_dict = {}

    for i in range(1,len(data)):
        token = data[i]
        if token in skills:
            try:
                skill_dict[token].add(i)
            except:
                skill_dict[token] = {i}
            
    for i in skill_dict:
        skill_dict[i] = len(skill_dict[i])
        
    return skill_dict

In [143]:
job_descr_dict = calc_skill_freq(processed_text)

Some Companies have mentioned the required skills in job position and some in some description <br>
Lets take a look at the skills mentioned in Job description column, then we will add them to get skills in demand

In [144]:
# remove punctuation present in job position column
def remove_punctuation(df):
    for p in ['/', ',', '(', ')', '-', '|', '&', '_', '.', '“', '”', ':']:
        df['Job_position'] = df['Job_position'].apply(lambda x: str(x).replace(p,' '))

    return df

In [145]:
df = remove_punctuation(df)

In [146]:
# analyzing stopwords from custom stopwords list
f = open("stopwords.txt","r",) 
stopwords=[]

for x in f:
    stopwords.append(x)
    
f.close()

In [147]:
for i in stopwords:
    stopwords = i.split(',')

In [148]:
for i in range(0, len(stopwords)):
    stopwords[i] = stopwords[i].replace("'","")
    stopwords[i] = stopwords[i].replace(" ","")
    stopwords[i] = stopwords[i].lower()

In [149]:
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', 'her', 'hers', 'herself', 'it', 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', 'should', 'no

In [150]:
# removing stopwords from the Job_position column
job_role = list(df['Job_position'].apply(lambda x: ' '.join([word for word in x.lower().split() if word not in (stopwords)])))

In [151]:
df['Job_position'] = df['Job_position'].apply(lambda x: ' '.join([word for word in x.lower().split() if word in (skills)]))
df['Job_position'] = df['Job_position'].where(df['Job_position'] != '', 'na')

In [152]:
job_role = word_tokenize(str(job_role))

In [153]:
# Now calculate the frequency of a particular skill mentioned in job role
job_role_dict = calc_skill_freq(job_role)

Below we first pass all the elements of the first dictionary into the third one and then pass the second dictionary 
into the third. This will replace the duplicate keys of the first dictionary. <br>
More info : (https://www.geeksforgeeks.org/python-merging-two-dictionaries/)

In [154]:
skills_dict = {**job_role_dict, **job_descr_dict}

Lets save this dictionary for now it will be useful for EDA

In [155]:
import pickle
skill_file = open('skill_dictionary', 'wb') 
pickle.dump(skills_dict, skill_file) 
skill_file.close() 

In [156]:
# now create new column for each skill with value equal to the frequency of that skill occurring in that particular cell
def calc_freq(df):
    for key in list(skills_dict.keys()):
        if skills_dict[key] > 15:
            skill_calc = []
            for i in range(0,len(df)):
                count = 0
                
                # here we are counting frquency from both requirements and Job position column
                for word in df['requirements'][i].lower().split() :
                    if key in df['Job_position'][i].lower().split():
                        count += 1
                    if key == word:
                        count += 1
                        skill_calc.append(count)
                    else:
                        skill_calc.append(0)
                        
            df = pd.concat([df, pd.DataFrame(skill_calc, columns=[key])], axis=1)
            # all the missing values should be filled with zero as they dont contain that particular skill 
            df[key] = df[key].fillna(0)
    return df
df = calc_freq(df)

As skills from job position and description were added its possible some of them dont appear in description, their frequency wiil be zero so we 
must drop them

In [157]:
# remove columns with constant values
df = df.loc[:, (df != df.iloc[0]).any()] 

In [158]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1124 entries, 0 to 1123
Data columns (total 89 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   Job_position        1124 non-null   object  
 1   Company             1124 non-null   object  
 2   Location            1124 non-null   object  
 3   requirements        1124 non-null   object  
 4   rating              1124 non-null   float64 
 5   experience          1124 non-null   object  
 6   posting_frequency   1124 non-null   float64 
 7   avg_yearly_sal      1124 non-null   float64 
 8   income_cat          1124 non-null   category
 9   net_experience      1124 non-null   int32   
 10  education_level     1124 non-null   object  
 11  bachelor            1124 non-null   uint8   
 12  masters             1124 non-null   uint8   
 13  na                  1124 non-null   uint8   
 14  secondary           1124 non-null   uint8   
 15  job_title           1124 non-null   ob

In [159]:
df.to_csv('./data_prepared.csv', index=False)